In [1]:
from qkeras import *
from tensorflow.keras.layers import Input, AveragePooling2D, Flatten, Softmax, Add, ZeroPadding2D, MaxPooling2D
from tensorflow.keras.backend import random_uniform as rand_tensor
import numpy as np
from qkeras import *
import math
from tensorflow import keras

c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def findMax(InArray, ph_start, pw_start, PKH, PKW):
    results = np.zeros((XN, YC))
    results -= math.inf
    for i in range(ph_start, ph_start+PKH):
        for j in range(pw_start, pw_start+PKW):
            if i >=0 and j>=0 and i < XH and j < XW:
                #cand = InArray[:,i,j,:]
                #results = np.maximum(results, cand)
                for n in range(XN):
                    for c in range(YC):
                        results[n][c] = max(results[n][c], InArray[n][i][j][c])
    return results

In [3]:
def findAvg(InArray, ph_start, pw_start, PKH, PKW):
    results = np.zeros((XN, YC))
    cnt = 0
    for i in range(ph_start, ph_start+PKH):
        for j in range(pw_start, pw_start+PKW):
            if i >=0 and j>=0 and i < XH and j < XW:
                results += InArray[:,i,j,:]
                cnt += 1
    return results/cnt

In [4]:
def findOutlen(inLen, filter_size, stride, padding_type, dilation=1):    
    adjusted_filter_size = filter_size + (filter_size-1)*(dilation-1)
    out_adj = inLen if padding_type=="same" else inLen-adjusted_filter_size+1
    return (out_adj+stride-1)//stride

In [5]:
Widths = [3, 10, 32, 60, 100]
Heights = [3, 10, 32, 60, 100]
pSize = [(1,1), (2,2), (3,3)]
pStride = [(1,1), (1,2), (2,1), (2,2), (2,3), (3,2), (3,3)]
mode = ["same", "valid"]

In [13]:
def myPooling2D(pool_type, InArray, Pool_size, Stride, padding_type):
    assert len(Pool_size)==2 and len(Stride)==2, f"{len(Pool_size)}, {len(Stride)}"
    assert padding_type in {"same", "valid"}
    assert pool_type in {"max", "avg"}

    XN, XH, XW, YC = InArray.shape
    PKH, PKW = Pool_size
    PSH, PSW = Stride

    # PYH = findOutlen(XH, PKH, PSH, padding_type)
    # PYW = findOutlen(XW, PKW, PSW, padding_type)

    if padding_type=="same":
        PYH = (XH+PSH-1)//PSH
        PYW = (XW+PSW-1)//PSW
    else:
        PYH = (XH-PKH+PSH)//PSH
        PYW = (XW-PKW+PSW)//PSW


    outArray = np.zeros((XN, PYH, PYW, YC))

    p_st, q_st = 0, 0
    if padding_type == "same":
        p_st = max((PSH*(PYH-1)+PKH-XH)//2, 0)
        q_st = max((PSW*(PYW-1)+PKW-XW)//2, 0)


    for n in range(XN):
        for c in range(YC):
            for iyh in range(PYH):
                for iyw in range(PYW):
                    ph_start, pw_start = PSH*iyh-p_st, PSW*iyw-q_st
                    ph_end, pw_end = PSH*iyh-p_st+PKH-1, PSW*iyw-q_st+PKW-1
                    
                    if pool_type=='max' :
                        result = -math.inf
                        for ixh in range(ph_end, ph_end-PKH,-1):
                            for ixw in range(pw_end, pw_end-PKW,-1):
                                if ixh >=0 and ixw>=0 and ixh < XH and ixw < XW:
                                    result = max(result, InArray[n][ixh][ixw][c])
                        
                        outArray[n,iyh,iyw,c] = result
                    else:

                        result = 0
                        cnt = 0
                        for ixh in range(ph_end, ph_end-PKH,-1):
                            for ixw in range(pw_end, pw_end-PKW,-1):
                                if ixh >=0 and ixw>=0 and ixh < XH and ixw < XW:
                                    result += InArray[n,ixh,ixw,c]
                                    cnt += 1
                        
                        outArray[n,iyh,iyw,c] = result/cnt
            
    return outArray


n = 1
c = 64
for w in Widths:
    for h in Heights:
        for size in pSize:
            for stride in pStride:
                for m in mode:
                    assert size[0]==size[1], f"pooling size must be square!"
                    x_in = tf.random.uniform(shape=(n, h, w, c), minval=-100, maxval=100, dtype=tf.int32)
                    input_shape = x_in.shape[1:-1] + (c,)
                
                    x = Input(input_shape, name='input')
                    x1 = MaxPooling2D(size, strides=stride, padding=m)(x)
                    model1 = Model(inputs=[x], outputs=[x1])                        
                    y_ref = model1(x_in).numpy()
                    y_np = myPooling2D("max", x_in.numpy(), size, stride, m)
                
                    assert y_ref.all() == y_np.all(), f"maxpool error! shape = ({m}, {size}, {stride}, {n}, {h}, {w}, {c}), {y_ref.shape}, {y_np.shape}, {x_in.numpy()}, {y_ref}, {y_np}"
                    
                    x2 = AveragePooling2D(size, strides=stride, padding=m)(x)
                    model2 = Model(inputs=[x], outputs=[x2])
                    y_ref = model2(x_in).numpy()
                    y_np = myPooling2D("avg", x_in.numpy(), size, stride, m)
                    assert y_ref.all() == y_np.all(), f"avgpool error! shape = ({m}, {size}, {stride}, {n}, {h}, {w}, {c}), {y_ref.shape}, {y_np.shape}, {x_in.numpy()}, {y_ref}, {y_np}"
                    